In [14]:
# %pip install ensemble_boxes

In [15]:
from ensemble_boxes import *
import pandas as pd
import numpy as np
import os

In [16]:
# dataset_path='../dataset'
csv_path='submission_cat.csv'
submission_df=pd.read_csv(csv_path)
submission_df.head()

,PredictionString,image_id
0,0 0.124515265 495.5461 647.56836 558.3712 759....,test/0000.jpg
1,0 0.93211544 639.43536 651.9728 683.84436 840....,test/0001.jpg
2,0 0.962425 882.3553 465.2519 1021.49457 634.34...,test/0002.jpg
3,0 0.07220997 1.7291985 14.298782 473.93475 101...,test/0003.jpg
4,0 0.8401868 430.48795 410.08987 660.095 571.94...,test/0004.jpg


In [17]:
img_id=submission_df['image_id'].tolist()
# img_id[:5]
# os.path.join(dataset_path,img_id[0])
pred_str=submission_df['PredictionString'].tolist()
# pred_str[:5]
print(len(pred_str))


4871


In [18]:
iou_thr=0.65
skip_box_thr=0.0001
sigma=0.1

ensemble_str=[]
file_path=[]
for i,img in enumerate(img_id):
# for img_out in pred_str:
	img_out=pred_str[i]
	boxes_list=[]
	scores_list=[]
	labels_list=[]
	
	if pd.isnull(img_out): continue

	img_out=np.reshape(img_out.split(),(-1,6)).tolist()

	for o in img_out:
		label=int(o[0])
		score=float(o[1])
		box=list(map(float,[o[2],o[3],o[4],o[5]]))
		box=list(map(lambda x:x/1024,box))
		boxes_list.append(box)
		scores_list.append(score)
		labels_list.append(label)
	
	# boxes, scores, labels = nms([boxes_list], [scores_list], [labels_list], iou_thr=iou_thr)
	boxes, scores, labels = soft_nms([boxes_list],[scores_list], [labels_list], iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
	# boxes, scores, labels = non_maximum_weighted([boxes_list],[scores_list], [labels_list], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
	# boxes, scores, labels = weighted_boxes_fusion([boxes_list],[scores_list], [labels_list], iou_thr=iou_thr, skip_box_thr=skip_box_thr)
	temp_str=''
	for box,score,label in zip(boxes,scores,labels):
		temp_str+=str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * 1024) + ' ' + str(box[1] * 1024) + ' ' + str(box[2] * 1024) + ' ' + str(box[3] * 1024) + ' '
	ensemble_str.append(temp_str)
	file_path.append(img)




Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 2 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed 1 boxes with zero area!
Warning. Removed

In [19]:
submission = pd.DataFrame()
submission['PredictionString'] = ensemble_str
submission['image_id'] = file_path
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,0 0.52490234375 329.5 639.0 353.0 683.0 0 0.12...,test/0000.jpg
1,0 0.93211544 639.43536 651.9728 683.84436 840....,test/0001.jpg
2,0 0.962425 882.3553 465.2519 1021.49457 634.34...,test/0002.jpg
3,0 0.07220997 1.7291985 14.298782 473.93475 101...,test/0003.jpg
4,0 0.93701171875 192.75 251.75 873.0 778.0 0 0....,test/0004.jpg
